# Proyecto precios casas (continuación)
## Segunda parte

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas_profiling

Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
import seaborn as sns
import numpy as np
from plotly import express as px
from matplotlib import pyplot as plt
from matplotlib import gridspec 
from pandas_profiling import ProfileReport
import geocoder
import time
import folium
pd.set_option('display.float_format', lambda x: '%.2f' % x )

## ETL 
Extract
Transform
Load

### Extraction

In [3]:
data = pd.read_csv('../data/kc_house_data.csv')

In [4]:
print('La base de datos tiene {} filas y {} columnas'.format(data.shape[0],data.shape[1]))

La base de datos tiene 21613 filas y 21 columnas


### Transformation

In [5]:
data.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [6]:
data['date'] = pd.to_datetime(data['date'], format = '%Y-%m-%d').dt.date
data['yr_built']= pd.to_datetime(data['yr_built'], format = '%Y').dt.year
data['yr_renovated'] = data['yr_renovated'].apply(lambda x: pd.to_datetime(x, format ='%Y') if x >0 else x )

In [7]:
data['id'] = data['id'].astype(str)
data['zipcode'] = data['zipcode'].astype(str)

In [8]:
data.dtypes

id                object
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated      object
zipcode           object
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

### Creando nuevas columnas

#### House_age

In [9]:
data.loc[data['yr_built']>=1990,'house_age'] = 'new_house' 
data.loc[data['yr_built']<1990,'house_age'] = 'old_house'

#### Dormitory_type

In [10]:
data.loc[data['bedrooms']<=1, 'dormitory_type'] = 'studio'
data.loc[data['bedrooms']==2, 'dormitory_type'] = 'apartment'
data.loc[data['bedrooms']>2, 'dormitory_type'] = 'house'

#### Condition_type

In [11]:
data.loc[data['condition']<=2, 'condition_type'] = 'bad'
data.loc[data['condition'].isin([3,4]), 'condition_type'] = 'regular'
data.loc[data['condition']== 5, 'condition_type'] = 'good'

#### Price_tier

In [12]:

data['price_tier'] = data['price'].apply(lambda x: 'tier 1' if x <= 321950 else
                                                      'tier 2' if (x > 321950) & (x <= 450000) else
                                                      'tier 3' if (x > 450000) & (x <= 645000) else
                                                      'tier 4')

#### Precio por pie cuadrado

In [13]:
data['price/sqft'] = data['price']/data['sqft_living']

#### Address, neighbourhood and city (API)

In [14]:
# for i in range(df_test.shape[0]):
# #     geolocator = Nominatim(user_agent='ErnestoPerez'+str(i))
# #     df_test.loc[i,'lat'], df_test.loc[i,'long']
#     geoloc = geocoder.osm([df_test.loc[i,'lat'], df_test.loc[i,'long']], method='reverse')
#     try:
#         df_test.loc[i,'house_number'] = geoloc.raw['address']['house_number']
#     except:
#         None
#     try:
#         df_test.loc[i,'road'] = geoloc.raw['address']['road']
#     except:
#         None
#     try:
#         df_test.loc[i,'neighbourhood'] = geoloc.raw['address']['neighbourhood']
#     except:
#         None
#     try:
#          df_test.loc[i,'city'] =geoloc.raw['address']['city']
#     except:
#         None
#     time.sleep(2)
#     if i % 150 == 0: 
#         print('Voy en la {}-ésima iteración'.format(i))

# Exploratory Descriptive Analysis (EDA)

In [15]:
prof = ProfileReport(data.drop(columns =['id','zipcode','yr_renovated']))
prof.to_file(output_file='data-EDA.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Validación de hipótesis

In [16]:
from IPython.display import Image
Image(url= "Precios_Casas.png", width=700, height=700)

In [17]:
att_num = data.select_dtypes(include = ['int64','float64'])

In [20]:
media = pd.DataFrame(att_num.apply(np.mean))
mediana = pd.DataFrame(att_num.apply(np.median))
std = pd.DataFrame(att_num.apply(np.std))
maximo = pd.DataFrame(att_num.apply(np.max))
minimo = pd.DataFrame(att_num.apply(np.min))

In [21]:
df_EDA = pd.concat([minimo,media,mediana,maximo,std], axis = 1)

In [22]:
df_EDA

,0,0,0,0,0
price,75000.00,540088.14,450000.00,7700000.00,367118.70
bedrooms,0.00,3.37,3.00,33.00,0.93
bathrooms,0.00,2.11,2.25,8.00,0.77
sqft_living,290.00,2079.90,1910.00,13540.00,918.42
sqft_lot,520.00,15106.97,7618.00,1651359.00,41419.55
floors,1.00,1.49,1.50,3.50,0.54
waterfront,0.00,0.01,0.00,1.00,0.09
view,0.00,0.23,0.00,4.00,0.77
condition,1.00,3.41,3.00,5.00,0.65
grade,1.00,7.66,7.00,13.00,1.18


In [23]:
df_EDA.columns = ['Mínimo','Media','Mediana','Máximo','std']

In [24]:
df_EDA

,Mínimo,Media,Mediana,Máximo,std
price,75000.00,540088.14,450000.00,7700000.00,367118.70
bedrooms,0.00,3.37,3.00,33.00,0.93
bathrooms,0.00,2.11,2.25,8.00,0.77
sqft_living,290.00,2079.90,1910.00,13540.00,918.42
sqft_lot,520.00,15106.97,7618.00,1651359.00,41419.55
floors,1.00,1.49,1.50,3.50,0.54
waterfront,0.00,0.01,0.00,1.00,0.09
view,0.00,0.23,0.00,4.00,0.77
condition,1.00,3.41,3.00,5.00,0.65
grade,1.00,7.66,7.00,13.00,1.18


In [25]:
data['lat'].mean()

47.56005251931704

In [26]:
df_test = data.head(100)

In [27]:
import folium
from folium.plugins import MarkerCluster

mapa = folium.Map(location=[df_test['lat'].mean(), df_test['long'].mean()], zoom_start=9)
marker_cluster = MarkerCluster().add_to(mapa)

In [29]:
for nombre, fila in df_test.iterrows():
    folium.Marker([fila['lat'],fila['long']],
                 popup = 'Propiedad Vendida en ${}, en {}.Características: {} habitaciones, {} baños, constuida en {}, área de {} pies cuadrados. Precio por pie cuadrado: {}'.format(
                 fila['price'],
                 fila['date'],
                 fila['bedrooms'],
                 fila['bathrooms'],
                 fila['yr_built'], 
                 fila['sqft_living'], 
                 fila['price/sqft'])
).add_to(marker_cluster)
 

In [30]:
mapa

In [35]:
data['yr_built'].min()

1900